# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [28]:
import pickle
import keras
# TODO: Implement load the data here.

training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [29]:
import numpy as np

# TODO: Implement data normalization here.
X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))
X_train = X_train - 0.5
# X_train = X_train - np.mean(X_train) + 0.2

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [33]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(128, input_dim=32*32*3, name='hidden1'),
    Activation('relu'),
    Dense(128, name='hidden2'),
    Activation('relu'),
    Dense(43, name='output'),
    Activation('softmax'),
])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [36]:
# TODO: Compile and train the model here.
from keras.utils import np_utils
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train.reshape((len(X_train), 32*32*3)), np_utils.to_categorical(y_train))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Epoch 1/10
39209/39209 [==============================] - 6s - loss: 1.3238 - acc: 0.6243     
Epoch 2/10
39209/39209 [==============================] - 6s - loss: 0.5806 - acc: 0.8271     
Epoch 3/10
39209/39209 [==============================] - 6s - loss: 0.4379 - acc: 0.8699     
Epoch 4/10
39209/39209 [==============================] - 6s - loss: 0.3863 - acc: 0.8844     
Epoch 5/10
39209/39209 [==============================] - 6s - loss: 0.3424 - acc: 0.8968     
Epoch 6/10
39209/39209 [==============================] - 6s - loss: 0.3201 - acc: 0.9051     
Epoch 7/10
39209/39209 [==============================] - 6s - loss: 0.2933 - acc: 0.9118     
Epoch 8/10
39209/39209 [==============================] - 6s - loss: 0.2809 - acc: 0.9147     
Epoch 9/10
39209/39209 [==============================] - 6s - loss: 0.2578 - acc: 0.9213     
Epoch 10/10
39209/39209 [==============================] - 6s - loss: 0.2333 - acc: 0.9296     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [38]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.
from sklearn.cross_validation import train_test_split

model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

def format(X, y):
    return X.reshape((len(X), 32*32*3)), np_utils.to_categorical(y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

X, y = format(X_train, y_train)
history = model.fit(X_train.reshape((len(X_train), 32*32*3)), np_utils.to_categorical(y_train), nb_epoch=2, validation_data=(format(X_val, y_val)))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 5s - loss: 0.2415 - acc: 0.9282 - val_loss: 0.2062 - val_acc: 0.9346
Epoch 2/2
29406/29406 [==============================] - 5s - loss: 0.2157 - acc: 0.9343 - val_loss: 0.2017 - val_acc: 0.9448


**Validation Accuracy**: 0.9488

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [40]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

nb_filters = 32

model = Sequential([
    Convolution2D(nb_filters, 3, 3,
                        border_mode='valid',
                        input_shape=(32, 32, 3)),
    Flatten(),
    Dense(128, name='hidden1'),
    Activation('relu'),
    Dense(128, name='hidden2'),
    Activation('relu'),
    Dense(43, name='output'),
    Activation('softmax'),
])


model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, np_utils.to_categorical(y_train), nb_epoch=2, validation_data=(X_val, np_utils.to_categorical(y_val)))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 13s - loss: 0.6896 - acc: 0.8109 - val_loss: 0.2931 - val_acc: 0.9148
Epoch 2/2
29406/29406 [==============================] - 12s - loss: 0.2079 - acc: 0.9427 - val_loss: 0.2329 - val_acc: 0.9412


**Validation Accuracy**: 0.9412

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [42]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.

model = Sequential([
    Convolution2D(nb_filters, 3, 3,
                        border_mode='valid',
                        input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, name='hidden1'),
    Activation('relu'),
    Dense(128, name='hidden2'),
    Activation('relu'),
    Dense(43, name='output'),
    Activation('softmax'),
])


model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, np_utils.to_categorical(y_train), nb_epoch=2, validation_data=(X_val, np_utils.to_categorical(y_val)))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 7s - loss: 0.8181 - acc: 0.7757 - val_loss: 0.2348 - val_acc: 0.9388
Epoch 2/2
29406/29406 [==============================] - 7s - loss: 0.1677 - acc: 0.9544 - val_loss: 0.1818 - val_acc: 0.9466


**Validation Accuracy**: 0.9466

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [43]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.
model = Sequential([
    Convolution2D(nb_filters, 3, 3,
                        border_mode='valid',
                        input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, name='hidden1'),
    Activation('relu'),
    Dense(128, name='hidden2'),
    Activation('relu'),
    Dense(43, name='output'),
    Activation('softmax'),
])


model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, np_utils.to_categorical(y_train), nb_epoch=2, validation_data=(X_val, np_utils.to_categorical(y_val)))


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 7s - loss: 0.9134 - acc: 0.7419 - val_loss: 0.2796 - val_acc: 0.9231
Epoch 2/2
29406/29406 [==============================] - 7s - loss: 0.2330 - acc: 0.9325 - val_loss: 0.1687 - val_acc: 0.9539


**Validation Accuracy**: 0.9539

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [45]:
model = Sequential([
    Convolution2D(nb_filters, 3, 3,
                        border_mode='valid',
                        input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(128, name='hidden1'),
    Activation('relu'),
    Dense(128, name='hidden2'),
    Activation('relu'),
    Dense(43, name='output'),
    Activation('softmax'),
])


model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, np_utils.to_categorical(y_train), nb_epoch=10, validation_data=(X_val, np_utils.to_categorical(y_val)))



Train on 29406 samples, validate on 9803 samples
Epoch 1/10
29406/29406 [==============================] - 7s - loss: 0.9273 - acc: 0.7436 - val_loss: 0.2893 - val_acc: 0.9193
Epoch 2/10
29406/29406 [==============================] - 7s - loss: 0.2394 - acc: 0.9304 - val_loss: 0.1495 - val_acc: 0.9616
Epoch 3/10
29406/29406 [==============================] - 7s - loss: 0.1634 - acc: 0.9519 - val_loss: 0.1416 - val_acc: 0.9635
Epoch 4/10
29406/29406 [==============================] - 7s - loss: 0.1253 - acc: 0.9622 - val_loss: 0.1245 - val_acc: 0.9648
Epoch 5/10
29406/29406 [==============================] - 7s - loss: 0.1055 - acc: 0.9690 - val_loss: 0.1373 - val_acc: 0.9640
Epoch 6/10
29406/29406 [==============================] - 7s - loss: 0.0868 - acc: 0.9733 - val_loss: 0.1216 - val_acc: 0.9684
Epoch 7/10
29406/29406 [==============================] - 7s - loss: 0.0946 - acc: 0.9717 - val_loss: 0.0975 - val_acc: 0.9764
Epoch 8/10
29406/29406 [==============================] - 7s -

**Best Validation Accuracy:** 0.9743

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [51]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

print(model.evaluate(X_test, Y_test))
print(model.metrics_names)

12576/12630 [============================>.] - ETA: 0s[0.66990808787698886, 0.89825811569216885]
['loss', 'acc']


**Test Accuracy:** 0.89825811569216885

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.